In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

from langgraph.graph import StateGraph , START , END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage , AIMessage , SystemMessage
from typing import TypedDict , Annotated
from pydantic import BaseModel , Field

### Tutorial Overview: Text Analysis Pipeline
In this tutorial, we'll demonstrate the power of LangGraph by building a multi-step text analysis pipeline. Our use case will focus on processing a given text through three key stages:

Text Classification: We'll categorize the input text into predefined categories (e.g., News, Blog, Research, or Other). \
Entity Extraction: We'll identify and extract key entities such as persons, organizations, and locations from the text. \
Text Summarization: Finally, we'll generate a concise summary of the input text. \
Title : We will also generate the title of the talk \
Content : We will join each one of them to create a blog  \
This pipeline showcases how LangGraph can be used to create a modular, extensible workflow for natural language processing tasks. By the end of this tutorial, you'll understand how to construct a graph-based application that can be easily modified or expanded for various text analysis needs.

In [2]:
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    text_classification: Annotated[str, None]
    entity_extraction: Annotated[dict[str, list[str]], None]
    summary: Annotated[str, None]
    title: Annotated[str, None]
    content: Annotated[str, None]


In [ ]:
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0 , max_completion_tokens=500)

In [4]:
from typing import Literal
from langgraph.graph import StateGraph, END

# --- Pydantic Schemas and LLMs ---

class TextClassification(BaseModel):
    """
    Classify the text into one of the following categories: News, Blog, Research, or Other.
    """
    category: Literal["News", "Blog", "Research", "Other"] = Field(description="The category of the text")

text_classification_llm = model.with_structured_output(TextClassification)

class EntityExtraction(BaseModel):
    """
    Extract entities from the text. Return a dictionary with keys 'person', 'organisation', and 'location'.
    Each key should map to a list of the corresponding entities found in the text.
    Example: {"person": ["John Doe"], "organisation": ["OpenAI"], "location": ["San Francisco", "India"]}
    If no entities are found, return an empty dictionary. If one key doesn't have a value, return an empty list for that key.
    """
    entities: dict[str, list[str]] = Field(
        description="A dictionary with keys 'person', 'organisation', and 'location', each mapping to a list of extracted entities."
    )

entity_extraction_llm = model.with_structured_output(EntityExtraction)

class TextSummarization(BaseModel):
    """
    Summarize the text in a concise manner.
    """
    summary: str = Field(description="The summary of the text")

text_summarization_llm = model.with_structured_output(TextSummarization)

class TitleGeneration(BaseModel):
    """
    Generate the title of the text.
    """
    title: str = Field(description="The title of the text")

title_generation_llm = model.with_structured_output(TitleGeneration)

class ContentGeneration(BaseModel):
    """
    Join the content of the text above and generate the content of the text.
    """
    content: str = Field(description="The content of the text")

content_generation_llm = model.with_structured_output(ContentGeneration)



In [ ]:
# --- LangGraph Node Functions ---

def classify_text(state):
    print('-> Calling text classification agent ->')

    text_classification_system_prompt = """
    You are a text classification agent. You will be given a user query and you will need to classify the text into one of the following categories: News, Blog, Research, or Other.

    Respond in json format with the following keys:

    category: The category of the text. Must be one of "News", "Blog", "Research", or "Other".
    """

    human_message = [msg for msg in state["messages"] if isinstance(msg, HumanMessage)]
    ai_message = [msg for msg in state["messages"] if isinstance(msg, AIMessage)]
    system_message = [SystemMessage(content=text_classification_system_prompt)]

    messages = system_message + ai_message + human_message

    result = text_classification_llm.invoke(messages)

    # We only update the text_classification, not messages. The router will use the category.
    return {
        "text_classification": result.category
    }


def extract_entities(state):
    print("-> Calling entity extraction agent ->")
    entity_extraction_system_prompt = """
    You are an entity extraction agent. Extract entities from the provided text. 
    Return a dictionary with keys 'person', 'organisation', and 'location'.
    Each key should map to a list of the corresponding entities found in the text.
    Example: {"person": ["John Doe"], "organisation": ["OpenAI"], "location": ["San Francisco", "India"]}
    If no entities are found, return an empty dictionary. If one key doesn't have a value, return an empty list for that key.
    Respond in JSON format.
    """
    human_message = [msg for msg in state["messages"] if isinstance(msg, HumanMessage)]
    ai_message = [msg for msg in state["messages"] if isinstance(msg, AIMessage)]
    system_message = [SystemMessage(content=entity_extraction_system_prompt)]
    messages = system_message + ai_message + human_message
    result = entity_extraction_llm.invoke(messages)
    # We only update the entity_extraction, not messages. The router will use the extracted entities.
    return {
        "entity_extraction": result.entities
    }

def summarize_text(state):
    print("-> Calling text summarization agent ->")
    human_message = [msg for msg in state["messages"] if isinstance(msg, HumanMessage)]
    ai_message = [msg for msg in state["messages"] if isinstance(msg, AIMessage)]
    system_message = [SystemMessage(content=text_summarization_system_prompt)]
    messages = system_message + ai_message + human_message
    result = text_summarization_llm.invoke(messages)
    return {"summary": result.summary}
    text = state["messages"][-1].content if state["messages"] else ""
    summarization_prompt = (
        "You are a text summarization agent. Summarize the following text in 2-3 sentences, "
        "focusing on the main points and omitting unnecessary details. "
        "Be concise and clear.\n\n"
        f"Text:\n{text}"
    )
    result = text_summarization_llm.invoke([HumanMessage(content=summarization_prompt)])
    return {"summary": result.summary}

def generate_title(state):
    print("🏷️ Generating title...")
    text = state["messages"][-1].content if state["messages"] else ""
    result = title_generation_llm.invoke([HumanMessage(content=text)])
    return {"title": result.title}

def generate_content(state):
    print("📰 Generating content...")
    # Compose content from previous steps
    summary = state.get("summary", "")
    entities = state.get("entity_extraction", {})
    category = state.get("text_classification", "")
    title = state.get("title", "")
    # Compose a prompt for content generation
    prompt = (
        f"Title: {title}\n"
        f"Category: {category}\n"
        f"Summary: {summary}\n"
        f"Entities: {entities}\n"
        "Write a blog post using the above information."
    )
    result = content_generation_llm.invoke([HumanMessage(content=prompt)])
    return {"content": result.content}

In [ ]:
# --- LangGraph Workflow Construction ---

# Workflow
# classify_text -> extract_entities -> summarize_text -> generate_title -> generate_content -> END

graph = StateGraph(GraphState)

# Add nodes for each step
graph.add_node("classify_text", classify_text)
graph.add_node("extract_entities", extract_entities)
graph.add_node("summarize_text", summarize_text)
graph.add_node("generate_title", generate_title)
graph.add_node("generate_content", generate_content)

# Define the workflow edges
graph.add_edge("classify_text", "extract_entities")
graph.add_edge("extract_entities", "summarize_text")
graph.add_edge("summarize_text", "generate_title")
graph.add_edge("generate_title", "generate_content")
graph.add_edge("generate_content", END)

# Set the entry point
graph.set_entry_point("classify_text")

# Compile the workflow app
awesome_langgraph_workflow = graph.compile()


In [ ]:

# --- Example: Run the workflow on a sample paragraph ---

# Example input paragraph (longer version)
example_paragraph = (
    "Artificial intelligence (AI) is rapidly transforming industries across the globe. "
    "From healthcare to finance, AI-powered solutions are enabling organizations to automate tasks, "
    "gain insights from data, and improve decision-making. As technology advances, ethical considerations "
    "and responsible AI development are becoming increasingly important to ensure positive societal impact. "
    "In recent years, AI has also made significant strides in areas such as natural language processing, "
    "computer vision, and robotics, leading to new applications that were previously unimaginable. "
    "For example, AI-driven diagnostic tools are helping doctors detect diseases earlier and with greater accuracy, "
    "while financial institutions use AI algorithms to detect fraud and manage risk. "
    "Despite these advancements, challenges remain, including concerns about data privacy, algorithmic bias, "
    "and the potential displacement of jobs. Addressing these issues requires collaboration between technologists, "
    "policymakers, and society at large to create frameworks that promote transparency, fairness, and accountability. "
    "Ultimately, the future of AI will depend on our ability to harness its power responsibly and for the benefit of all."
)

# Prepare the initial state for the workflow
from langchain_core.messages import HumanMessage

initial_state = {
    "messages": [HumanMessage(content=example_paragraph)]
}

# Run the workflow
result = awesome_langgraph_workflow.invoke(initial_state)

# Print the results
print("=== Workflow Output ===")
for key, value in result.items():
    print(f"{key}: {value}")